## Import Libraries

In [1]:
import pandas as pd
import numpy as np

# **Import scrapped CSV**

In [2]:
df_fight_stats =pd.read_csv('../scrape_ufc_stats-main/ufc_fight_stats.csv')
df_event_details =pd.read_csv('../scrape_ufc_stats-main/ufc_event_details.csv')
df_fighter_tott = pd.read_csv('../scrape_ufc_stats-main/ufc_fighter_tott.csv')
df_results = pd.read_csv('../scrape_ufc_stats-main/ufc_fight_results.csv')
df_fighter_details =pd.read_csv('../scrape_ufc_stats-main/ufc_fighter_details.csv')

In [3]:
import re
def extract_rounds(text):
  if text == "No Time Limit":
    return [0]
  else:
    result = re.findall(r'\((.*?)\)', text)
    result = list(map(int,result[0].split("-")))
    return result

def handle_result_df(df):
  WEIGHTCLASSES_my = ['Lightweight', 'Welterweight', 'Middleweight', 'Featherweight','Heavyweigh', 'Light Heavyweight', 'Bantamweight', 'Flyweight', 'Strawweight', 'Open Weight', 'Catch Weight']
  df['Winner'] = df.apply(lambda row: 'NC' if row['OUTCOME'] == 'NC/NC' else 'DRAW' if row['OUTCOME'] == 'D/D' else row['BOUT'].split(' vs. ')[(row['OUTCOME'].split('/')).index('W')], axis=1)
  WeightClasses_df = df['WEIGHTCLASS']
  result=[]
  women = []
  Title = []
  for bout in WeightClasses_df:
    bout_low = bout.lower()
    women.append(1) if 'women' in bout_low else women.append(0)
    Title.append(1) if 'title' in bout_low else Title.append(0)
    find = False
    for classes in WEIGHTCLASSES_my:   
      if classes.lower() in bout_low.lower():
        result.append(classes)
        find = True      
        break
    if not find:
      result.append('Open Weight')
  
  df.loc[:,"WeightClass"] = result
  df.loc[:,"Tittle"] = Title
  df.loc[:,"Women_fight"] = women
  df['TimeSec'] = df['TIME'].apply(lambda x: (datetime.strptime(x, '%M:%S') - datetime(1900,1,1)).total_seconds()).astype(int)
  df['TIME FORMAT'] = df['TIME FORMAT'].apply(lambda x: extract_rounds(str(x)))
  df_results['TotalTime'] = df_results.apply(lambda row: sum(row['TIME FORMAT'][0:int(row['ROUND'])-1])*60 + row['TimeSec'],axis=1)
  df = df.drop(['OUTCOME','WEIGHTCLASS','TimeSec'],axis=1)
  df_results = df_results.merge(df_event_details[['EVENT','DATE']],how='left', left_on='EVENT', right_on='EVENT')
  df = df[['DATE','EVENT', 'BOUT', 'Winner', 'WeightClass', 'Tittle', 'Women_fight','METHOD','TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS', 'URL']]
  df['BOUT'] = df['BOUT'].str.split().str.join(' ')
  df['EVENT'] = df['EVENT'].str.split().str.join(' ')
  return df

# **handle duplicated fighters name**

In [4]:
df_fighter_tott.shape

(3926, 7)

check each fighter if ha has fight in UFC (DWCS not included) if not drop fighters - not solved duplicated values

In [5]:
df_fighter_tott['Has_fight'] = df_fighter_tott['FIGHTER'].apply(lambda x: x in list(df_fight_stats['FIGHTER']))

In [6]:
df_fighter_tott[df_fighter_tott['Has_fight'] == False]

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,URL,Has_fight
0,Tom Aaron,--,155 lbs.,--,NaN,"Jul 13, 1978",http://ufcstats.com/fighter-details/93fe7332d1...,False
4,Hiroyuki Abe,"5' 6""",145 lbs.,--,Orthodox,--,http://ufcstats.com/fighter-details/c0ed7b2081...,False
9,Daniel Acacio,"5' 8""",180 lbs.,--,Orthodox,"Dec 27, 1977",http://ufcstats.com/fighter-details/1c5879330d...,False
12,Anthony Adams,"6' 1""",185 lbs.,"76""",Orthodox,"Jan 13, 1988",http://ufcstats.com/fighter-details/a77633a989...,False
19,Fabio Aguiar,"6' 0""",185 lbs.,--,NaN,"Feb 10, 1988",http://ufcstats.com/fighter-details/501821d7fb...,False
...,...,...,...,...,...,...,...,...
3916,Gaston Bolanos,--,145 lbs.,--,NaN,"Sep 14, 1992",http://ufcstats.com/fighter-details/96924b0019...,False
3920,Shara Magomedov,"6' 2""",170 lbs.,--,NaN,"May 16, 1994",http://ufcstats.com/fighter-details/06734ca9d8...,False
3922,Francisco Prado,"5' 10""",155 lbs.,--,NaN,"Jun 16, 2002",http://ufcstats.com/fighter-details/3920d0cc28...,False
3924,Elves Brenner,"5' 9""",155 lbs.,--,NaN,"Sep 27, 1997",http://ufcstats.com/fighter-details/48a9a12878...,False


In [7]:
df_fighter_tott = df_fighter_tott.loc[df_fighter_tott['Has_fight']==True,:]

In [8]:
df_fighter_tott

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,URL,Has_fight
1,Danny Abbadi,"5' 11""",155 lbs.,--,Orthodox,"Jul 03, 1983",http://ufcstats.com/fighter-details/15df64c02b...,True
2,David Abbott,"6' 0""",265 lbs.,--,Switch,--,http://ufcstats.com/fighter-details/b361180739...,True
3,Shamil Abdurakhimov,"6' 3""",235 lbs.,"76""",Orthodox,"Sep 02, 1981",http://ufcstats.com/fighter-details/2f5cbecbbe...,True
5,Daichi Abe,"5' 11""",170 lbs.,"71""",Orthodox,"Nov 27, 1991",http://ufcstats.com/fighter-details/5140122c3e...,True
6,Papy Abedi,"5' 11""",185 lbs.,--,Southpaw,"Jun 30, 1978",http://ufcstats.com/fighter-details/c9f6385af6...,True
...,...,...,...,...,...,...,...,...
3917,Steven Koslow,--,125 lbs.,--,NaN,--,http://ufcstats.com/fighter-details/8451233947...,True
3918,Luan Lacerda,"5' 7""",135 lbs.,--,NaN,"Jan 07, 1993",http://ufcstats.com/fighter-details/6fc506e109...,True
3919,Nick Fiore,"5' 11""",155 lbs.,--,NaN,"Dec 10, 1997",http://ufcstats.com/fighter-details/036e96c1c1...,True
3921,Melquizael Costa,"5' 10""",155 lbs.,--,NaN,"Sep 14, 1996",http://ufcstats.com/fighter-details/20bccc9bb4...,True


In [9]:
df_fighter_tott['WEIGHT'] = pd.to_numeric(df_fighter_tott['WEIGHT'].str.replace('lbs.','',regex=True), errors='coerce')
df_fighter_tott['REACH'] = pd.to_numeric(df_fighter_tott['REACH'].str.replace('"','',regex=True), errors='coerce')

In [10]:
df_fighter_tott['HEIGHT'] = df_fighter_tott['HEIGHT'].replace('--',np.nan)
df_fighter_tott[['ft','inch']] = df_fighter_tott['HEIGHT'].str.extractall(r"(\d+)").unstack().astype(float)
df_fighter_tott['HEIGHT'] = df_fighter_tott['ft']*12 + df_fighter_tott['inch']

In [11]:
df_fighter_tott = df_fighter_tott.drop(['ft', 'inch'], axis =1)

**drop fighters dupplicated fighters which has no start in ufc**

In [12]:
df_fighter_tott.drop(df_fighter_tott[df_fighter_tott['URL']=='http://ufcstats.com/fighter-details/c8661e204c66f325'].index, inplace=True)
df_fighter_tott.drop(df_fighter_tott[df_fighter_tott['URL']=='http://ufcstats.com/fighter-details/3a28e1e641366308'].index, inplace=True)
df_fighter_tott.drop(df_fighter_tott[df_fighter_tott['URL']=='http://ufcstats.com/fighter-details/d52ef694108f8235'].index, inplace=True)

find fighters with same names and put nickname to one of them

In [13]:
duplicated_fighters_index = list(df_fighter_tott[df_fighter_tott['FIGHTER'].duplicated()].index)[0]


In [14]:
duplicated_fighters_index

3109

In [15]:
df_fighter_tott[df_fighter_tott['FIGHTER'] == df_fighter_tott.loc[duplicated_fighters_index,'FIGHTER']]

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,URL,Has_fight
3108,Bruno Silva,64.0,125.0,65.0,Orthodox,"Mar 16, 1990",http://ufcstats.com/fighter-details/294aa73dbf...,True
3109,Bruno Silva,72.0,185.0,74.0,Orthodox,"Jul 13, 1989",http://ufcstats.com/fighter-details/12ebd7d157...,True


In [16]:
name_dup_fighter = df_fighter_tott.loc[duplicated_fighters_index,'FIGHTER']
url_dup_fighter = df_fighter_tott.loc[duplicated_fighters_index,'URL']
url_dup_fighter

'http://ufcstats.com/fighter-details/12ebd7d157e91701'

In [17]:
df_fighter_details.head()

,FIRST,LAST,NICKNAME,URL
0,Tom,Aaron,NaN,http://ufcstats.com/fighter-details/93fe7332d1...
1,Danny,Abbadi,The Assassin,http://ufcstats.com/fighter-details/15df64c02b...
2,Nariman,Abbasov,Bayraktar,http://ufcstats.com/fighter-details/59a9d6dac6...
3,David,Abbott,Tank,http://ufcstats.com/fighter-details/b361180739...
4,Hamdy,Abdelwahab,The Hammer,http://ufcstats.com/fighter-details/3329d692ae...


In [18]:
nickname=df_fighter_details.loc[df_fighter_details['URL']==url_dup_fighter, 'NICKNAME'].values[0]
weight_class ='Middleweight'
nickname

'Blindado'

**update name in fighter tott df**

In [19]:
df_fighter_tott.loc[df_fighter_tott['URL'] == url_dup_fighter, 'FIGHTER'] = df_fighter_tott.loc[df_fighter_tott['URL'] == url_dup_fighter, 'FIGHTER'] + ' ' + nickname

In [20]:
df_fighter_tott.loc[df_fighter_tott['URL'] == url_dup_fighter, 'FIGHTER']

3109    Bruno Silva Blindado
Name: FIGHTER, dtype: object

In [21]:
fights_list = df_results[(df_results['BOUT'].str.contains(name_dup_fighter)) & (df_results['WEIGHTCLASS'].str.contains(weight_class))]['BOUT'].str.replace(" ", "").tolist()
fights_list

['BrunoSilvavs.GeraldMeerschaert',
 'AlexPereiravs.BrunoSilva',
 'JordanWrightvs.BrunoSilva',
 'AndrewSanchezvs.BrunoSilva',
 'WellingtonTurmanvs.BrunoSilva']

**update name in fight stats df**

In [22]:
df_fight_stats.loc[(df_fight_stats['BOUT'].str.replace(" ", "").isin(fights_list)) & (df_fight_stats['FIGHTER']==name_dup_fighter),'FIGHTER'] = name_dup_fighter + ' ' + nickname

In [23]:
df_fight_stats.loc[(df_fight_stats['BOUT'].str.replace(" ", "").isin(fights_list))]

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
946,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 1,Bruno Silva Blindado,0.0,10 of 26,38%,19 of 38,0 of 0,---,0.0,0.0,0:00,5 of 17,3 of 6,2 of 3,9 of 25,1 of 1,0 of 0
947,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 2,Bruno Silva Blindado,0.0,12 of 41,29%,12 of 41,0 of 0,---,0.0,0.0,0:00,8 of 32,2 of 7,2 of 2,12 of 41,0 of 0,0 of 0
948,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 3,Bruno Silva Blindado,0.0,7 of 14,50%,7 of 14,0 of 0,---,0.0,0.0,0:01,4 of 11,1 of 1,2 of 2,6 of 13,1 of 1,0 of 0
949,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 1,Gerald Meerschaert,0.0,12 of 38,31%,20 of 52,1 of 1,100%,0.0,0.0,1:55,8 of 33,2 of 2,2 of 3,6 of 25,0 of 0,6 of 13
950,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 2,Gerald Meerschaert,0.0,19 of 51,37%,19 of 51,0 of 2,0%,0.0,0.0,0:03,8 of 38,6 of 8,5 of 5,19 of 50,0 of 0,0 of 1
951,UFC Fight Night: Vera vs. Cruz,Bruno Silva vs. Gerald Meerschaert,Round 3,Gerald Meerschaert,1.0,15 of 25,60%,22 of 33,0 of 3,0%,1.0,0.0,0:14,12 of 21,1 of 2,2 of 2,11 of 20,0 of 0,4 of 5
2056,UFC Fight Night: Santos vs. Ankalaev,Alex Pereira vs. Bruno Silva,Round 1,Alex Pereira,0.0,27 of 51,52%,31 of 55,0 of 0,---,0.0,0.0,0:00,10 of 28,11 of 16,6 of 7,23 of 45,4 of 6,0 of 0
2057,UFC Fight Night: Santos vs. Ankalaev,Alex Pereira vs. Bruno Silva,Round 2,Alex Pereira,0.0,33 of 58,56%,53 of 79,0 of 0,---,0.0,0.0,0:07,19 of 41,13 of 16,1 of 1,30 of 53,3 of 4,0 of 1
2058,UFC Fight Night: Santos vs. Ankalaev,Alex Pereira vs. Bruno Silva,Round 3,Alex Pereira,0.0,48 of 70,68%,81 of 105,0 of 0,---,0.0,0.0,1:10,31 of 51,17 of 18,0 of 1,33 of 53,15 of 17,0 of 0
2059,UFC Fight Night: Santos vs. Ankalaev,Alex Pereira vs. Bruno Silva,Round 1,Bruno Silva Blindado,0.0,16 of 46,34%,19 of 49,1 of 1,100%,0.0,0.0,0:48,10 of 40,2 of 2,4 of 4,13 of 42,3 of 4,0 of 0


# **Fight_stats_details df**

In [24]:
df_fight_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32374 entries, 0 to 32373
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   EVENT       32374 non-null  object 
 1   BOUT        32374 non-null  object 
 2   ROUND       32332 non-null  object 
 3   FIGHTER     32332 non-null  object 
 4   KD          32332 non-null  float64
 5   SIG.STR.    32332 non-null  object 
 6   SIG.STR. %  32332 non-null  object 
 7   TOTAL STR.  32332 non-null  object 
 8   TD          32332 non-null  object 
 9   TD %        32332 non-null  object 
 10  SUB.ATT     32332 non-null  float64
 11  REV.        32332 non-null  float64
 12  CTRL        32332 non-null  object 
 13  HEAD        32332 non-null  object 
 14  BODY        32332 non-null  object 
 15  LEG         32332 non-null  object 
 16  DISTANCE    32332 non-null  object 
 17  CLINCH      32332 non-null  object 
 18  GROUND      32332 non-null  object 
dtypes: float64(3), object(16)

In [25]:
df_fight_stats.head()

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
0,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Round 1,Derrick Lewis,0.0,0 of 3,0%,0 of 3,0 of 0,---,0.0,0.0,0:00,0 of 2,0 of 1,0 of 0,0 of 3,0 of 0,0 of 0
1,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Round 1,Serghei Spivac,0.0,12 of 21,57%,36 of 54,6 of 8,75%,1.0,0.0,2:32,12 of 21,0 of 0,0 of 0,0 of 3,0 of 0,12 of 18
2,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Round 1,Da Woon Jung,0.0,5 of 15,33%,9 of 20,0 of 4,0%,0.0,0.0,2:06,5 of 15,0 of 0,0 of 0,2 of 10,3 of 5,0 of 0
3,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Round 2,Da Woon Jung,0.0,6 of 12,50%,23 of 29,0 of 0,---,0.0,0.0,2:16,2 of 8,3 of 3,1 of 1,3 of 7,3 of 5,0 of 0
4,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Round 3,Da Woon Jung,0.0,7 of 25,28%,11 of 29,1 of 1,100%,0.0,0.0,1:26,4 of 19,3 of 6,0 of 0,2 of 19,5 of 6,0 of 0


**drop rows where are not fighter filled**

In [26]:
df_fight_stats = df_fight_stats[df_fight_stats['FIGHTER'].notna()]

**splits stats by 'of' to attemp, landed**

In [27]:
cols_to_split = [col for col in list(df_fight_stats.columns) if ' of ' in str(df_fight_stats.loc[0,col])]

In [28]:
for col in cols_to_split:
  print(col + '_landed')
  df_fight_stats[[col + '_landed', col + '_attemps']] = (df_fight_stats[col].str.split(' of ', expand=True))
  df_fight_stats[[col + '_landed', col + '_attemps']] = df_fight_stats[[col + '_landed', col + '_attemps']].fillna(0)
  df_fight_stats[[col + '_landed', col + '_attemps']] = df_fight_stats[[col + '_landed', col + '_attemps']].astype(int)

SIG.STR._landed
TOTAL STR._landed
TD_landed
HEAD_landed
BODY_landed
LEG_landed
DISTANCE_landed
CLINCH_landed
GROUND_landed


In [29]:
df_fight_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32332 entries, 0 to 32373
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EVENT               32332 non-null  object 
 1   BOUT                32332 non-null  object 
 2   ROUND               32332 non-null  object 
 3   FIGHTER             32332 non-null  object 
 4   KD                  32332 non-null  float64
 5   SIG.STR.            32332 non-null  object 
 6   SIG.STR. %          32332 non-null  object 
 7   TOTAL STR.          32332 non-null  object 
 8   TD                  32332 non-null  object 
 9   TD %                32332 non-null  object 
 10  SUB.ATT             32332 non-null  float64
 11  REV.                32332 non-null  float64
 12  CTRL                32332 non-null  object 
 13  HEAD                32332 non-null  object 
 14  BODY                32332 non-null  object 
 15  LEG                 32332 non-null  object 
 16  DIST

**drop original columns + columns with %**

In [30]:
cols_to_del = cols_to_split + ['SIG.STR. %', 'TD %']
df_fight_stats = df_fight_stats.drop(cols_to_del, axis = 1)

In [31]:
df_fight_stats.isna().any()

EVENT                 False
BOUT                  False
ROUND                 False
FIGHTER               False
KD                    False
SUB.ATT               False
REV.                  False
CTRL                  False
SIG.STR._landed       False
SIG.STR._attemps      False
TOTAL STR._landed     False
TOTAL STR._attemps    False
TD_landed             False
TD_attemps            False
HEAD_landed           False
HEAD_attemps          False
BODY_landed           False
BODY_attemps          False
LEG_landed            False
LEG_attemps           False
DISTANCE_landed       False
DISTANCE_attemps      False
CLINCH_landed         False
CLINCH_attemps        False
GROUND_landed         False
GROUND_attemps        False
dtype: bool

In [32]:
df_fight_stats.head(3)

,EVENT,BOUT,ROUND,FIGHTER,KD,SUB.ATT,REV.,CTRL,SIG.STR._landed,SIG.STR._attemps,...,BODY_landed,BODY_attemps,LEG_landed,LEG_attemps,DISTANCE_landed,DISTANCE_attemps,CLINCH_landed,CLINCH_attemps,GROUND_landed,GROUND_attemps
0,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Round 1,Derrick Lewis,0.0,0.0,0.0,0:00,0,3,...,0,1,0,0,0,3,0,0,0,0
1,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Round 1,Serghei Spivac,0.0,1.0,0.0,2:32,12,21,...,0,0,0,0,0,3,0,0,12,18
2,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Round 1,Da Woon Jung,0.0,0.0,0.0,2:06,5,15,...,0,0,0,0,2,10,3,5,0,0


**create agg df**

In [33]:
df_fight_stats_agg = df_fight_stats.copy()

In [34]:
df_fight_stats_agg = df_fight_stats_agg.drop(['ROUND'], axis = 1)

In [35]:
df_fight_stats_agg.head(10)

,EVENT,BOUT,FIGHTER,KD,SUB.ATT,REV.,CTRL,SIG.STR._landed,SIG.STR._attemps,TOTAL STR._landed,...,BODY_landed,BODY_attemps,LEG_landed,LEG_attemps,DISTANCE_landed,DISTANCE_attemps,CLINCH_landed,CLINCH_attemps,GROUND_landed,GROUND_attemps
0,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Derrick Lewis,0.0,0.0,0.0,0:00,0,3,0,...,0,1,0,0,0,3,0,0,0,0
1,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Serghei Spivac,0.0,1.0,0.0,2:32,12,21,36,...,0,0,0,0,0,3,0,0,12,18
2,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Da Woon Jung,0.0,0.0,0.0,2:06,5,15,9,...,0,0,0,0,2,10,3,5,0,0
3,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Da Woon Jung,0.0,0.0,0.0,2:16,6,12,23,...,3,3,1,1,3,7,3,5,0,0
4,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Da Woon Jung,0.0,0.0,0.0,1:26,7,25,11,...,3,6,0,0,2,19,5,6,0,0
5,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Devin Clark,0.0,0.0,0.0,2:09,10,15,42,...,1,1,5,5,6,9,3,3,1,3
6,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Devin Clark,0.0,0.0,0.0,1:25,10,18,24,...,1,1,5,5,6,13,4,5,0,0
7,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Devin Clark,0.0,0.0,0.0,1:28,19,37,30,...,3,4,2,3,14,30,4,6,1,1
8,UFC Fight Night: Lewis vs. Spivac,Marcin Tybura vs. Blagoy Ivanov,Marcin Tybura,0.0,0.0,0.0,0:00,22,45,22,...,10,12,7,7,22,45,0,0,0,0
9,UFC Fight Night: Lewis vs. Spivac,Marcin Tybura vs. Blagoy Ivanov,Marcin Tybura,0.0,0.0,0.0,0:00,17,57,17,...,9,12,1,1,17,57,0,0,0,0


**handle control time value** 

In [36]:
df_fight_stats_agg['CTRL'] = df_fight_stats_agg['CTRL'].replace("--", "0:00")

In [37]:
from datetime import datetime, timedelta

In [38]:
df_fight_stats_agg['CTRL'] = df_fight_stats_agg['CTRL'].apply(lambda x: (datetime.strptime(x, '%M:%S') - datetime(1900,1,1)).total_seconds()).astype(int)

In [39]:
df_fight_stats_agg['CTRL'].describe()

count    32332.000000
mean        56.378572
std         74.218739
min          0.000000
25%          0.000000
50%         20.000000
75%         90.000000
max        300.000000
Name: CTRL, dtype: float64

In [40]:
df_fight_stats_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32332 entries, 0 to 32373
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EVENT               32332 non-null  object 
 1   BOUT                32332 non-null  object 
 2   FIGHTER             32332 non-null  object 
 3   KD                  32332 non-null  float64
 4   SUB.ATT             32332 non-null  float64
 5   REV.                32332 non-null  float64
 6   CTRL                32332 non-null  int32  
 7   SIG.STR._landed     32332 non-null  int32  
 8   SIG.STR._attemps    32332 non-null  int32  
 9   TOTAL STR._landed   32332 non-null  int32  
 10  TOTAL STR._attemps  32332 non-null  int32  
 11  TD_landed           32332 non-null  int32  
 12  TD_attemps          32332 non-null  int32  
 13  HEAD_landed         32332 non-null  int32  
 14  HEAD_attemps        32332 non-null  int32  
 15  BODY_landed         32332 non-null  int32  
 16  BODY

**AGG df by bout**

In [41]:
df_total = df_fight_stats_agg.groupby(['EVENT','BOUT','FIGHTER'],as_index=False).sum()

In [42]:
df_total.isna().any()

EVENT                 False
BOUT                  False
FIGHTER               False
KD                    False
SUB.ATT               False
REV.                  False
CTRL                  False
SIG.STR._landed       False
SIG.STR._attemps      False
TOTAL STR._landed     False
TOTAL STR._attemps    False
TD_landed             False
TD_attemps            False
HEAD_landed           False
HEAD_attemps          False
BODY_landed           False
BODY_attemps          False
LEG_landed            False
LEG_attemps           False
DISTANCE_landed       False
DISTANCE_attemps      False
CLINCH_landed         False
CLINCH_attemps        False
GROUND_landed         False
GROUND_attemps        False
dtype: bool

In [43]:
df_total['BOUT'] = df_total['BOUT'].str.strip()

In [44]:
df_total['BOUT']

0         Jason MacDonald vs. Ed Herman
1         Jason MacDonald vs. Ed Herman
2         Kendall Grove vs. Chris Price
3         Kendall Grove vs. Chris Price
4         Marcus Davis vs. Forrest Petz
                      ...              
13851    Rory Markham vs. Brodie Farber
13852    Shannon Gugerty vs. Dale Hartt
13853    Shannon Gugerty vs. Dale Hartt
13854    Tim Credeur vs. Cale Yarbrough
13855    Tim Credeur vs. Cale Yarbrough
Name: BOUT, Length: 13856, dtype: object

**split bout to fighter oponnent columns**

In [45]:
df_total['OPPONENT'] = df_total.apply(lambda row: row['BOUT'].split(' vs. ')[0] if row['FIGHTER'] == row['BOUT'].split(' vs. ')[1] else row['BOUT'].split(' vs. ')[1], axis=1)

In [46]:
df_total.shape

(13856, 26)

**přidání datumu k eventu a vypočítání věku fightera**

In [47]:
df_total = df_total.merge(df_event_details[['EVENT','DATE']],how='left', left_on='EVENT', right_on='EVENT')
df_total['DATE'] = df_total['DATE'].apply(pd.to_datetime)

In [48]:
df_total.shape

(13856, 27)

In [49]:
df_fighter_tott['DOB'] = df_fighter_tott['DOB'] = pd.to_datetime(df_fighter_tott['DOB'], errors='coerce')

**přidání parametrů fightera**

In [50]:
df_fighter_tott

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,URL,Has_fight
1,Danny Abbadi,71.0,155.0,NaN,Orthodox,1983-07-03,http://ufcstats.com/fighter-details/15df64c02b...,True
2,David Abbott,72.0,265.0,NaN,Switch,NaT,http://ufcstats.com/fighter-details/b361180739...,True
3,Shamil Abdurakhimov,75.0,235.0,76.0,Orthodox,1981-09-02,http://ufcstats.com/fighter-details/2f5cbecbbe...,True
5,Daichi Abe,71.0,170.0,71.0,Orthodox,1991-11-27,http://ufcstats.com/fighter-details/5140122c3e...,True
6,Papy Abedi,71.0,185.0,NaN,Southpaw,1978-06-30,http://ufcstats.com/fighter-details/c9f6385af6...,True
...,...,...,...,...,...,...,...,...
3917,Steven Koslow,NaN,125.0,NaN,NaN,NaT,http://ufcstats.com/fighter-details/8451233947...,True
3918,Luan Lacerda,67.0,135.0,NaN,NaN,1993-01-07,http://ufcstats.com/fighter-details/6fc506e109...,True
3919,Nick Fiore,71.0,155.0,NaN,NaN,1997-12-10,http://ufcstats.com/fighter-details/036e96c1c1...,True
3921,Melquizael Costa,70.0,155.0,NaN,NaN,1996-09-14,http://ufcstats.com/fighter-details/20bccc9bb4...,True


In [51]:
df_total = df_total.merge(df_fighter_tott[['FIGHTER', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE','DOB']], left_on='FIGHTER', right_on='FIGHTER', how='left')

In [52]:
#df_total = df_total.merge(df_fighter_tott[['FIGHTER', 'HEIGHT', 'WEIGHT', 'REACH', 'STANCE','DOB']], left_on='OPPONENT', right_on='FIGHTER', suffixes=('_fighter','_opponent'))

In [53]:
df_total.shape

(13857, 32)

In [54]:
df_total.isna().any()

EVENT                 False
BOUT                  False
FIGHTER               False
KD                    False
SUB.ATT               False
REV.                  False
CTRL                  False
SIG.STR._landed       False
SIG.STR._attemps      False
TOTAL STR._landed     False
TOTAL STR._attemps    False
TD_landed             False
TD_attemps            False
HEAD_landed           False
HEAD_attemps          False
BODY_landed           False
BODY_attemps          False
LEG_landed            False
LEG_attemps           False
DISTANCE_landed       False
DISTANCE_attemps      False
CLINCH_landed         False
CLINCH_attemps        False
GROUND_landed         False
GROUND_attemps        False
OPPONENT              False
DATE                  False
HEIGHT                 True
WEIGHT                 True
REACH                  True
STANCE                 True
DOB                    True
dtype: bool

In [55]:
df_total['AGE'] = (pd.to_datetime(df_total['DATE']) - pd.to_datetime(df_total['DOB'])).astype('<m8[Y]')
#df_total['AGE_opponent'] = (pd.to_datetime(df_total['DATE']) - pd.to_datetime(df_total['DOB_opponent'])).astype('<m8[Y]')

In [56]:
#df_total = df_total.drop(['FIGHTER_opponent', 'DOB_fighter', 'DOB_opponent'],axis=1)
df_total = df_total.drop(['DOB'],axis=1)

In [57]:
df_total = df_total.sort_values(by=['DATE', 'BOUT'], ascending=True).reset_index(drop=True)

In [58]:
list_fighters = list(df_total['FIGHTER'])
#list_fighters = list(df_total['FIGHTER_fighter'])

**vypočítaní kolikátý zápas je to pro fightera**

In [59]:
#fight_no_fighter = [list_fighters[0:i].count(fighter)+1 for i, fighter in enumerate(list(df_total['FIGHTER_fighter']))]
#fight_no_opponent = [list_fighters[0:i].count(fighter)+1 for i, fighter in enumerate(list(df_total['OPPONENT']))]
fightNo_fighter = [list_fighters[0:i].count(fighter)+1 for i, fighter in enumerate(list(df_total['FIGHTER']))]

In [60]:
df_total['fightNo'] = fightNo_fighter
#df_total['fight_no_opponent'] = fight_no_opponent

In [61]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13857 entries, 0 to 13856
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   EVENT               13857 non-null  object        
 1   BOUT                13857 non-null  object        
 2   FIGHTER             13857 non-null  object        
 3   KD                  13857 non-null  float64       
 4   SUB.ATT             13857 non-null  float64       
 5   REV.                13857 non-null  float64       
 6   CTRL                13857 non-null  int32         
 7   SIG.STR._landed     13857 non-null  int32         
 8   SIG.STR._attemps    13857 non-null  int32         
 9   TOTAL STR._landed   13857 non-null  int32         
 10  TOTAL STR._attemps  13857 non-null  int32         
 11  TD_landed           13857 non-null  int32         
 12  TD_attemps          13857 non-null  int32         
 13  HEAD_landed         13857 non-null  int32     

In [62]:
df_total.isna().any()

EVENT                 False
BOUT                  False
FIGHTER               False
KD                    False
SUB.ATT               False
REV.                  False
CTRL                  False
SIG.STR._landed       False
SIG.STR._attemps      False
TOTAL STR._landed     False
TOTAL STR._attemps    False
TD_landed             False
TD_attemps            False
HEAD_landed           False
HEAD_attemps          False
BODY_landed           False
BODY_attemps          False
LEG_landed            False
LEG_attemps           False
DISTANCE_landed       False
DISTANCE_attemps      False
CLINCH_landed         False
CLINCH_attemps        False
GROUND_landed         False
GROUND_attemps        False
OPPONENT              False
DATE                  False
HEIGHT                 True
WEIGHT                 True
REACH                  True
STANCE                 True
AGE                    True
fightNo               False
dtype: bool

In [63]:
df_total.columns

Index(['EVENT', 'BOUT', 'FIGHTER', 'KD', 'SUB.ATT', 'REV.', 'CTRL',
       'SIG.STR._landed', 'SIG.STR._attemps', 'TOTAL STR._landed',
       'TOTAL STR._attemps', 'TD_landed', 'TD_attemps', 'HEAD_landed',
       'HEAD_attemps', 'BODY_landed', 'BODY_attemps', 'LEG_landed',
       'LEG_attemps', 'DISTANCE_landed', 'DISTANCE_attemps', 'CLINCH_landed',
       'CLINCH_attemps', 'GROUND_landed', 'GROUND_attemps', 'OPPONENT', 'DATE',
       'HEIGHT', 'WEIGHT', 'REACH', 'STANCE', 'AGE', 'fightNo'],
      dtype='object')

In [64]:
df_total = df_total.merge(df_total,how='left', left_on=['EVENT', 'BOUT', 'OPPONENT'], right_on=['EVENT', 'BOUT', 'FIGHTER'], suffixes=('_fighter', '_opponent'))

In [65]:
df_total.isna().sum()

EVENT                  0
BOUT                   0
FIGHTER_fighter        0
KD_fighter             0
SUB.ATT_fighter        0
                    ... 
WEIGHT_opponent       32
REACH_opponent      1392
STANCE_opponent      196
AGE_opponent         265
fightNo_opponent       9
Length: 64, dtype: int64

In [66]:
df_total = df_total.drop(['FIGHTER_opponent', 'OPPONENT_opponent', 'DATE_opponent'],axis=1)

In [67]:
df_total= df_total.rename(columns={'BOUT_fighter': "BOUT", "FIGHTER_fighter": "FIGHTER", "OPPONENT_fighter": "OPPONENT"})

In [68]:
df_total.shape

(13858, 61)

In [69]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13858 entries, 0 to 13857
Data columns (total 61 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   EVENT                        13858 non-null  object        
 1   BOUT                         13858 non-null  object        
 2   FIGHTER                      13858 non-null  object        
 3   KD_fighter                   13858 non-null  float64       
 4   SUB.ATT_fighter              13858 non-null  float64       
 5   REV._fighter                 13858 non-null  float64       
 6   CTRL_fighter                 13858 non-null  int32         
 7   SIG.STR._landed_fighter      13858 non-null  int32         
 8   SIG.STR._attemps_fighter     13858 non-null  int32         
 9   TOTAL STR._landed_fighter    13858 non-null  int32         
 10  TOTAL STR._attemps_fighter   13858 non-null  int32         
 11  TD_landed_fighter            13858 non-nu

In [70]:
df_total.isna().any()

EVENT               False
BOUT                False
FIGHTER             False
KD_fighter          False
SUB.ATT_fighter     False
                    ...  
WEIGHT_opponent      True
REACH_opponent       True
STANCE_opponent      True
AGE_opponent         True
fightNo_opponent     True
Length: 61, dtype: bool

In [71]:
df_results.head()

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,L/W,Heavyweight Bout,Submission,1,3:05,5 Rnd (5-5-5-5-5),Marc Goddard,Arm Triangle On Ground,http://ufcstats.com/fight-details/caeeb398f73d...
1,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,L/W,Light Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,Eric Colon 27 - 30.Chris Lee 27 - 30.David Let...,http://ufcstats.com/fight-details/7a1abc0230af...
2,UFC Fight Night: Lewis vs. Spivac,Marcin Tybura vs. Blagoy Ivanov,W/L,Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mark Smith,Junichiro Kamijo 27 - 30.Ben Cartlidge 28 - 29...,http://ufcstats.com/fight-details/8741e6a3a5a2...
3,UFC Fight Night: Lewis vs. Spivac,Dooho Choi vs. Kyle Nelson,D/D,Featherweight Bout,Decision - Majority,3,5:00,3 Rnd (5-5-5),Chris Tognoni,Adalaide Byrd 27 - 29.Eric Colon 28 - 28.Chris...,http://ufcstats.com/fight-details/01e886808600...
4,UFC Fight Night: Lewis vs. Spivac,Yusaku Kinoshita vs. Adam Fugitt,L/W,Welterweight Bout,KO/TKO,1,4:36,3 Rnd (5-5-5),Keith Peterson,Elbows to Head From Mount,http://ufcstats.com/fight-details/c63d8062c89c...


In [72]:
df_results['Winner'] = df_results.apply(lambda row: 'NC' if row['OUTCOME'] == 'NC/NC' else 'DRAW' if row['OUTCOME'] == 'D/D' else row['BOUT'].split(' vs. ')[(row['OUTCOME'].split('/')).index('W')], axis=1)

In [73]:
df_results.head(30)

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,Winner
0,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,L/W,Heavyweight Bout,Submission,1,3:05,5 Rnd (5-5-5-5-5),Marc Goddard,Arm Triangle On Ground,http://ufcstats.com/fight-details/caeeb398f73d...,Serghei Spivac
1,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,L/W,Light Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Keith Peterson,Eric Colon 27 - 30.Chris Lee 27 - 30.David Let...,http://ufcstats.com/fight-details/7a1abc0230af...,Devin Clark
2,UFC Fight Night: Lewis vs. Spivac,Marcin Tybura vs. Blagoy Ivanov,W/L,Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mark Smith,Junichiro Kamijo 27 - 30.Ben Cartlidge 28 - 29...,http://ufcstats.com/fight-details/8741e6a3a5a2...,Marcin Tybura
3,UFC Fight Night: Lewis vs. Spivac,Dooho Choi vs. Kyle Nelson,D/D,Featherweight Bout,Decision - Majority,3,5:00,3 Rnd (5-5-5),Chris Tognoni,Adalaide Byrd 27 - 29.Eric Colon 28 - 28.Chris...,http://ufcstats.com/fight-details/01e886808600...,DRAW
4,UFC Fight Night: Lewis vs. Spivac,Yusaku Kinoshita vs. Adam Fugitt,L/W,Welterweight Bout,KO/TKO,1,4:36,3 Rnd (5-5-5),Keith Peterson,Elbows to Head From Mount,http://ufcstats.com/fight-details/c63d8062c89c...,Adam Fugitt
5,UFC Fight Night: Lewis vs. Spivac,Jeka Saragih vs. Anshul Jubli,L/W,Lightweight Bout,KO/TKO,2,3:44,3 Rnd + OT (5-5-5-5),Marc Goddard,Elbows to Head From Mount,http://ufcstats.com/fight-details/b1b20c686dfe...,Anshul Jubli
6,UFC Fight Night: Lewis vs. Spivac,JeongYeong Lee vs. Yi Zha,W/L,Featherweight Bout,Decision - Split,3,5:00,3 Rnd + OT (5-5-5-5),Mark Smith,Jacob Montalvo 30 - 27.David Lethaby 28 - 29.J...,http://ufcstats.com/fight-details/765482eb58e8...,JeongYeong Lee
7,UFC Fight Night: Lewis vs. Spivac,Toshiomi Kazama vs. Rinya Nakamura,L/W,Bantamweight Bout,KO/TKO,1,0:33,3 Rnd + OT (5-5-5-5),Keith Peterson,Punch to Head At Distance,http://ufcstats.com/fight-details/a8067e65eb73...,Rinya Nakamura
8,UFC Fight Night: Lewis vs. Spivac,SeungGuk Choi vs. HyunSung Park,L/W,Flyweight Bout,Submission,3,3:11,3 Rnd + OT (5-5-5-5),Marc Goddard,Rear Naked Choke,http://ufcstats.com/fight-details/98094a500529...,HyunSung Park
9,UFC Fight Night: Lewis vs. Spivac,JunYong Park vs. Denis Tiuliulin,W/L,Middleweight Bout,Submission,1,4:05,3 Rnd (5-5-5),Mark Smith,Rear Naked Choke,http://ufcstats.com/fight-details/983fab408074...,JunYong Park


In [74]:
df_results.groupby('TIME FORMAT')['TIME FORMAT'].count().sort_values(ascending=False).head(50)

TIME FORMAT
3 Rnd (5-5-5)           6104
5 Rnd (5-5-5-5-5)        601
1 Rnd + OT (12-3)         88
No Time Limit             31
3 Rnd + OT (5-5-5-5)      30
1 Rnd (20)                25
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                12
1 Rnd (10)                 7
1 Rnd (12)                 6
1 Rnd + OT (30-5)          3
1 Rnd (18)                 2
1 Rnd + OT (15-3)          2
1 Rnd + OT (27-3)          1
1 Rnd + OT (30-3)          1
1 Rnd + OT (31-5)          1
1 Rnd + 2OT (24-3-3)       1
1 Rnd (30)                 1
Name: TIME FORMAT, dtype: int64

In [75]:
WEIGHTCLASSES_my = ['Lightweight', 'Welterweight', 'Middleweight', 'Featherweight','Heavyweigh', 'Light Heavyweight', 'Bantamweight', 'Flyweight', 'Strawweight', 'Open Weight', 'Catch Weight']

In [76]:
WeightClasses_df = df_results['WEIGHTCLASS']

In [77]:
WeightClasses_df

0             Heavyweight Bout
1       Light Heavyweight Bout
2             Heavyweight Bout
3           Featherweight Bout
4            Welterweight Bout
                 ...          
6945          Open Weight Bout
6946          Open Weight Bout
6947          Open Weight Bout
6948          Open Weight Bout
6949          Open Weight Bout
Name: WEIGHTCLASS, Length: 6950, dtype: object

In [78]:
result=[]
women = []
Title = []
for bout in WeightClasses_df:
  bout_low = bout.lower()
  women.append(1) if 'women' in bout_low else women.append(0)
  Title.append(1) if 'title' in bout_low else Title.append(0)
  find = False
  for classes in WEIGHTCLASSES_my:   
    if classes.lower() in bout_low.lower():
      result.append(classes)
      find = True      
      break
  if not find:
    result.append('Open Weight')
    print(classes, bout)

Catch Weight Ultimate Ultimate '96 Tournament Title Bout
Catch Weight UFC 10 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight UFC 8 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight Ultimate Ultimate '95 Tournament Title Bout
Catch Weight UFC 7 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight UFC 6 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight UFC 5 Tournament Title Bout
Catch Weight UFC Superfight Championship Bout
Catch Weight UFC 4 Tournament Title Bout
Catch Weight UFC 3 Tournament Title Bout
Catch Weight UFC 2 Tournament Title Bout


In [79]:
result[:10]

['Heavyweigh',
 'Heavyweigh',
 'Heavyweigh',
 'Featherweight',
 'Welterweight',
 'Lightweight',
 'Featherweight',
 'Bantamweight',
 'Flyweight',
 'Middleweight']

In [80]:
df_results.loc[:,"WeightClass"] = result
df_results.loc[:,"Tittle"] = Title
df_results.loc[:,"Women_fight"] = women


In [81]:
df_results.tail()

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,Winner,WeightClass,Tittle,Women_fight
6945,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,No Time Limit,John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...,Orlando Wiet,Open Weight,0,0
6946,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,No Time Limit,John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...,Frank Hamaker,Open Weight,0,0
6947,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,No Time Limit,John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...,Johnny Rhodes,Open Weight,0,0
6948,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,No Time Limit,John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...,Patrick Smith,Open Weight,0,0
6949,UFC 2: No Way Out,Scott Morris vs. Sean Daugherty,W/L,Open Weight Bout,Submission,1,0:20,No Time Limit,John McCarthy,Guillotine Choke From Mount,http://ufcstats.com/fight-details/4acab67848e7...,Scott Morris,Open Weight,0,0


In [82]:
df_results['TimeSec'] = df_results['TIME'].apply(lambda x: (datetime.strptime(x, '%M:%S') - datetime(1900,1,1)).total_seconds()).astype(int)

In [83]:
df_results.groupby('TIME FORMAT')['TIME FORMAT'].count().sort_values(ascending=False).head(50)

TIME FORMAT
3 Rnd (5-5-5)           6104
5 Rnd (5-5-5-5-5)        601
1 Rnd + OT (12-3)         88
No Time Limit             31
3 Rnd + OT (5-5-5-5)      30
1 Rnd (20)                25
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                12
1 Rnd (10)                 7
1 Rnd (12)                 6
1 Rnd + OT (30-5)          3
1 Rnd (18)                 2
1 Rnd + OT (15-3)          2
1 Rnd + OT (27-3)          1
1 Rnd + OT (30-3)          1
1 Rnd + OT (31-5)          1
1 Rnd + 2OT (24-3-3)       1
1 Rnd (30)                 1
Name: TIME FORMAT, dtype: int64

In [84]:
import re
def extract_rounds(text):
  if text == "No Time Limit":
    return [0]
  else:
    result = re.findall(r'\((.*?)\)', text)
    result = list(map(int,result[0].split("-")))
    return result

In [85]:
df_results['TIME FORMAT'] = df_results['TIME FORMAT'].apply(lambda x: extract_rounds(x))

In [86]:
df_results

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,Winner,WeightClass,Tittle,Women_fight,TimeSec
0,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,L/W,Heavyweight Bout,Submission,1,3:05,"[5, 5, 5, 5, 5]",Marc Goddard,Arm Triangle On Ground,http://ufcstats.com/fight-details/caeeb398f73d...,Serghei Spivac,Heavyweigh,0,0,185
1,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,L/W,Light Heavyweight Bout,Decision - Unanimous,3,5:00,"[5, 5, 5]",Keith Peterson,Eric Colon 27 - 30.Chris Lee 27 - 30.David Let...,http://ufcstats.com/fight-details/7a1abc0230af...,Devin Clark,Heavyweigh,0,0,300
2,UFC Fight Night: Lewis vs. Spivac,Marcin Tybura vs. Blagoy Ivanov,W/L,Heavyweight Bout,Decision - Unanimous,3,5:00,"[5, 5, 5]",Mark Smith,Junichiro Kamijo 27 - 30.Ben Cartlidge 28 - 29...,http://ufcstats.com/fight-details/8741e6a3a5a2...,Marcin Tybura,Heavyweigh,0,0,300
3,UFC Fight Night: Lewis vs. Spivac,Dooho Choi vs. Kyle Nelson,D/D,Featherweight Bout,Decision - Majority,3,5:00,"[5, 5, 5]",Chris Tognoni,Adalaide Byrd 27 - 29.Eric Colon 28 - 28.Chris...,http://ufcstats.com/fight-details/01e886808600...,DRAW,Featherweight,0,0,300
4,UFC Fight Night: Lewis vs. Spivac,Yusaku Kinoshita vs. Adam Fugitt,L/W,Welterweight Bout,KO/TKO,1,4:36,"[5, 5, 5]",Keith Peterson,Elbows to Head From Mount,http://ufcstats.com/fight-details/c63d8062c89c...,Adam Fugitt,Welterweight,0,0,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6945,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,W/L,Open Weight Bout,KO/TKO,1,2:50,[0],John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...,Orlando Wiet,Open Weight,0,0,170
6946,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,W/L,Open Weight Bout,Submission,1,4:52,[0],John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...,Frank Hamaker,Open Weight,0,0,292
6947,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,W/L,Open Weight Bout,KO/TKO,1,12:13,[0],John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...,Johnny Rhodes,Open Weight,0,0,733
6948,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,W/L,Open Weight Bout,Submission,1,0:58,[0],John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...,Patrick Smith,Open Weight,0,0,58


In [87]:
df_results['TIME FORMAT']

0       [5, 5, 5, 5, 5]
1             [5, 5, 5]
2             [5, 5, 5]
3             [5, 5, 5]
4             [5, 5, 5]
             ...       
6945                [0]
6946                [0]
6947                [0]
6948                [0]
6949                [0]
Name: TIME FORMAT, Length: 6950, dtype: object

In [88]:
df_results['TotalTime'] = df_results.apply(lambda row: sum(row['TIME FORMAT'][0:int(row['ROUND'])-1])*60 + row['TimeSec'],axis=1)

In [89]:
df_results['ROUND'].unique()

array([1, 3, 2, 5, 4], dtype=int64)

In [90]:
df_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6950 entries, 0 to 6949
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   EVENT        6950 non-null   object
 1   BOUT         6950 non-null   object
 2   OUTCOME      6950 non-null   object
 3   WEIGHTCLASS  6950 non-null   object
 4   METHOD       6950 non-null   object
 5   ROUND        6950 non-null   int64 
 6   TIME         6950 non-null   object
 7   TIME FORMAT  6950 non-null   object
 8   REFEREE      6918 non-null   object
 9   DETAILS      6877 non-null   object
 10  URL          6950 non-null   object
 11  Winner       6950 non-null   object
 12  WeightClass  6950 non-null   object
 13  Tittle       6950 non-null   int64 
 14  Women_fight  6950 non-null   int64 
 15  TimeSec      6950 non-null   int32 
 16  TotalTime    6950 non-null   int64 
dtypes: int32(1), int64(4), object(12)
memory usage: 896.0+ KB


In [91]:
df_results = df_results.drop(['OUTCOME','WEIGHTCLASS','TimeSec'],axis=1)

In [92]:
df_results.columns

Index(['EVENT', 'BOUT', 'METHOD', 'ROUND', 'TIME', 'TIME FORMAT', 'REFEREE',
       'DETAILS', 'URL', 'Winner', 'WeightClass', 'Tittle', 'Women_fight',
       'TotalTime'],
      dtype='object')

In [93]:
df_event_details

,EVENT,URL,DATE,LOCATION
0,UFC Fight Night: Lewis vs. Spivac,http://ufcstats.com/event-details/d26394fc0e8e...,"February 04, 2023","Las Vegas, Nevada, USA"
1,UFC 283: Teixeira vs. Hill,http://ufcstats.com/event-details/5717efc6f271...,"January 21, 2023","Rio de Janeiro, Rio de Janeiro, Brazil"
2,UFC Fight Night: Strickland vs. Imavov,http://ufcstats.com/event-details/2e2cdb6e9eb8...,"January 14, 2023","Las Vegas, Nevada, USA"
3,UFC Fight Night: Cannonier vs. Strickland,http://ufcstats.com/event-details/56ec58954158...,"December 17, 2022","Las Vegas, Nevada, USA"
4,UFC 282: Blachowicz vs. Ankalaev,http://ufcstats.com/event-details/f65a0eb902f9...,"December 10, 2022","Las Vegas, Nevada, USA"
...,...,...,...,...
628,UFC 6: Clash of the Titans,http://ufcstats.com/event-details/1c3f5e85b59e...,"July 14, 1995","Casper, Wyoming, USA"
629,UFC 5: The Return of the Beast,http://ufcstats.com/event-details/dedc3bb440d0...,"April 07, 1995","Charlotte, North Carolina, USA"
630,UFC 4: Revenge of the Warriors,http://ufcstats.com/event-details/b60391da771d...,"December 16, 1994","Tulsa, Oklahoma, USA"
631,UFC 3: The American Dream,http://ufcstats.com/event-details/1a49e0670dfa...,"September 09, 1994","Charlotte, North Carolina, USA"


In [94]:
df_results

,EVENT,BOUT,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL,Winner,WeightClass,Tittle,Women_fight,TotalTime
0,UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Submission,1,3:05,"[5, 5, 5, 5, 5]",Marc Goddard,Arm Triangle On Ground,http://ufcstats.com/fight-details/caeeb398f73d...,Serghei Spivac,Heavyweigh,0,0,185
1,UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Decision - Unanimous,3,5:00,"[5, 5, 5]",Keith Peterson,Eric Colon 27 - 30.Chris Lee 27 - 30.David Let...,http://ufcstats.com/fight-details/7a1abc0230af...,Devin Clark,Heavyweigh,0,0,900
2,UFC Fight Night: Lewis vs. Spivac,Marcin Tybura vs. Blagoy Ivanov,Decision - Unanimous,3,5:00,"[5, 5, 5]",Mark Smith,Junichiro Kamijo 27 - 30.Ben Cartlidge 28 - 29...,http://ufcstats.com/fight-details/8741e6a3a5a2...,Marcin Tybura,Heavyweigh,0,0,900
3,UFC Fight Night: Lewis vs. Spivac,Dooho Choi vs. Kyle Nelson,Decision - Majority,3,5:00,"[5, 5, 5]",Chris Tognoni,Adalaide Byrd 27 - 29.Eric Colon 28 - 28.Chris...,http://ufcstats.com/fight-details/01e886808600...,DRAW,Featherweight,0,0,900
4,UFC Fight Night: Lewis vs. Spivac,Yusaku Kinoshita vs. Adam Fugitt,KO/TKO,1,4:36,"[5, 5, 5]",Keith Peterson,Elbows to Head From Mount,http://ufcstats.com/fight-details/c63d8062c89c...,Adam Fugitt,Welterweight,0,0,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6945,UFC 2: No Way Out,Orlando Wiet vs. Robert Lucarelli,KO/TKO,1,2:50,[0],John McCarthy,toCorner Stoppage,http://ufcstats.com/fight-details/3b020d4914b4...,Orlando Wiet,Open Weight,0,0,170
6946,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,Submission,1,4:52,[0],John McCarthy,Keylock From Half Guard,http://ufcstats.com/fight-details/d917c8c7461b...,Frank Hamaker,Open Weight,0,0,292
6947,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,KO/TKO,1,12:13,[0],John McCarthy,Punches to Head From GuardSubmission to Strikes,http://ufcstats.com/fight-details/ccee020be2e8...,Johnny Rhodes,Open Weight,0,0,733
6948,UFC 2: No Way Out,Patrick Smith vs. Ray Wizard,Submission,1,0:58,[0],John McCarthy,Guillotine Choke Standing,http://ufcstats.com/fight-details/4b9ae533ccb3...,Patrick Smith,Open Weight,0,0,58


In [95]:
df_results['BOUT'] = df_results['BOUT'].str.split().str.join(' ')
df_results['EVENT'] = df_results['EVENT'].str.split().str.join(' ')

In [96]:
df_results = df_results.merge(df_event_details[['EVENT','DATE']],how='left', left_on='EVENT', right_on='EVENT')

In [97]:
df_results = df_results[['DATE','EVENT', 'BOUT', 'Winner', 'WeightClass', 'Tittle', 'Women_fight','METHOD','TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS', 'URL']]

In [98]:
df_results.head()

,DATE,EVENT,BOUT,Winner,WeightClass,Tittle,Women_fight,METHOD,TIME FORMAT,ROUND,TIME,TotalTime,REFEREE,DETAILS,URL
0,"February 04, 2023",UFC Fight Night: Lewis vs. Spivac,Derrick Lewis vs. Serghei Spivac,Serghei Spivac,Heavyweigh,0,0,Submission,"[5, 5, 5, 5, 5]",1,3:05,185,Marc Goddard,Arm Triangle On Ground,http://ufcstats.com/fight-details/caeeb398f73d...
1,"February 04, 2023",UFC Fight Night: Lewis vs. Spivac,Da Woon Jung vs. Devin Clark,Devin Clark,Heavyweigh,0,0,Decision - Unanimous,"[5, 5, 5]",3,5:00,900,Keith Peterson,Eric Colon 27 - 30.Chris Lee 27 - 30.David Let...,http://ufcstats.com/fight-details/7a1abc0230af...
2,"February 04, 2023",UFC Fight Night: Lewis vs. Spivac,Marcin Tybura vs. Blagoy Ivanov,Marcin Tybura,Heavyweigh,0,0,Decision - Unanimous,"[5, 5, 5]",3,5:00,900,Mark Smith,Junichiro Kamijo 27 - 30.Ben Cartlidge 28 - 29...,http://ufcstats.com/fight-details/8741e6a3a5a2...
3,"February 04, 2023",UFC Fight Night: Lewis vs. Spivac,Dooho Choi vs. Kyle Nelson,DRAW,Featherweight,0,0,Decision - Majority,"[5, 5, 5]",3,5:00,900,Chris Tognoni,Adalaide Byrd 27 - 29.Eric Colon 28 - 28.Chris...,http://ufcstats.com/fight-details/01e886808600...
4,"February 04, 2023",UFC Fight Night: Lewis vs. Spivac,Yusaku Kinoshita vs. Adam Fugitt,Adam Fugitt,Welterweight,0,0,KO/TKO,"[5, 5, 5]",1,4:36,276,Keith Peterson,Elbows to Head From Mount,http://ufcstats.com/fight-details/c63d8062c89c...


In [99]:
df_results['METHOD'].unique()

array(['Submission ', 'Decision - Unanimous ', 'Decision - Majority ',
       'KO/TKO ', 'Decision - Split ', "TKO - Doctor's Stoppage ",
       'Could Not Continue ', 'DQ ', 'Other ', 'Overturned '],
      dtype=object)

In [100]:
df_results[df_results['METHOD']=='KO/TKO ']['DETAILS'].unique()

array(['Elbows to Head From Mount ', 'Punch to Head At Distance ',
       'Punches to Head In Clinch ', 'Punches to Head At Distance ',
       'Punches to Head From Back Control ', 'Flying Knee Head ',
       'Knees to Body At Distance ', 'Kick to Body At Distance ',
       'Punches to Head From Guard ', 'Kick to Head At Distance ',
       'Punch to Head On Ground ', 'Punches to Head On Ground ',
       'Punch to Head In Clinch ', 'toCorner Stoppage',
       'Punches to Head From Half Guard ', 'Knee to Head At Distance ',
       'Kick to Head At DistanceWheel Kick', 'Spinning Back Kick Head ',
       'Kick to Body At DistanceFront Kick',
       'Elbows to Head From Half Guard ', 'Flying Knee Body ',
       'to At DistanceKnee injury', 'to At Distance ',
       'Kick to Leg At Distance ', 'Knee to Head In Clinch ',
       'Punches to Head From Mount ', 'Kick to Body From Bottom ',
       'Punch to Body At Distance ', 'Elbows to Head On Ground ',
       'Knees to Head In Clinch ', 'toKne

In [101]:
df_results[df_results['DETAILS']=='toSubmission to Strikes']

,DATE,EVENT,BOUT,Winner,WeightClass,Tittle,Women_fight,METHOD,TIME FORMAT,ROUND,TIME,TotalTime,REFEREE,DETAILS,URL
6897,"September 08, 1995",UFC 7: The Brawl in Buffalo,Onassis Parungao vs. Francesco Maturi,Onassis Parungao,Open Weight,0,0,KO/TKO,[20],1,5:26,326,John McCarthy,toSubmission to Strikes,http://ufcstats.com/fight-details/e4fe950846b5...


In [102]:
df_results[df_results['BOUT'].str.contains('Silva')]


,DATE,EVENT,BOUT,Winner,WeightClass,Tittle,Women_fight,METHOD,TIME FORMAT,ROUND,TIME,TotalTime,REFEREE,DETAILS,URL
60,"December 10, 2022",UFC 282: Blachowicz vs. Ankalaev,TJ Brown vs. Erik Silva,TJ Brown,Featherweight,0,0,Submission,"[5, 5, 5]",3,3:41,821,Mark Smith,Arm Triangle From Side Control,http://ufcstats.com/fight-details/2d9d966e0e8d...
86,"November 19, 2022",UFC Fight Night: Nzechukwu vs. Cutelaba,Natalia Silva vs. Tereza Bleda,Natalia Silva,Flyweight,0,1,KO/TKO,"[5, 5, 5]",3,1:27,687,Chris Tognoni,Spinning Back Kick Head,http://ufcstats.com/fight-details/33719a2d9c69...
97,"November 12, 2022",UFC 281: Adesanya vs. Pereira,Karolina Kowalkiewicz vs. Silvana Gomez Juarez,Karolina Kowalkiewicz,Strawweight,0,1,Decision - Unanimous,"[5, 5, 5]",3,5:00,900,Marc Goddard,Mike Bell 28 - 29.Sal D'amato 28 - 29.Dave Tir...,http://ufcstats.com/fight-details/ff2e4b411f69...
141,"October 01, 2022",UFC Fight Night: Dern vs. Yan,Joaquim Silva vs. Jesse Ronson,Joaquim Silva,Lightweight,0,0,KO/TKO,"[5, 5, 5]",2,3:08,488,Keith Peterson,Flying Knee Head,http://ufcstats.com/fight-details/ecc56a86a1ee...
194,"August 20, 2022",UFC 278: Usman vs. Edwards,Daniel Da Silva vs. Victor Altamirano,Victor Altamirano,Flyweight,0,0,KO/TKO,"[5, 5, 5]",1,3:39,219,Dave Seljestad,Punches to Head From Guard,http://ufcstats.com/fight-details/06d6cde74797...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6381,"May 27, 2006",UFC 60: Hughes vs Gracie,Brandon Vera vs. Assuerio Silva,Brandon Vera,Heavyweigh,0,0,Submission,"[5, 5, 5]",1,2:39,159,John McCarthy,Guillotine Choke From Bottom Guard,http://ufcstats.com/fight-details/c66bd50ed1e7...
6422,"January 16, 2006",UFC Fight Night 3,Tim Sylvia vs. Assuerio Silva,Tim Sylvia,Heavyweigh,0,0,Decision - Unanimous,"[5, 5, 5]",3,5:00,900,John McCarthy,Adalaide Byrd 27 - 30.Cecil Peoples 27 - 30.Da...,http://ufcstats.com/fight-details/537012aa6fb2...
6713,"April 14, 2000",UFC 25: Ultimate Japan 3,Tito Ortiz vs. Wanderlei Silva,Tito Ortiz,Heavyweigh,1,0,Decision - Unanimous,"[5, 5, 5, 5, 5]",5,5:00,1500,John McCarthy,47 - 49.46 - 49.46 - 49.,http://ufcstats.com/fight-details/0adfdf8f563b...
6752,"May 07, 1999",UFC 20: Battle for the Gold,Wanderlei Silva vs. Tony Petarra,Wanderlei Silva,Middleweight,0,0,KO/TKO,"[12, 3]",1,2:53,173,Mario Yamasaki,Knees to Head In Clinch,http://ufcstats.com/fight-details/c892b23ad8f2...


In [103]:
df_total

,EVENT,BOUT,FIGHTER,KD_fighter,SUB.ATT_fighter,REV._fighter,CTRL_fighter,SIG.STR._landed_fighter,SIG.STR._attemps_fighter,TOTAL STR._landed_fighter,...,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent,HEIGHT_opponent,WEIGHT_opponent,REACH_opponent,STANCE_opponent,AGE_opponent,fightNo_opponent
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,Frank Hamaker,0.0,3.0,1.0,0,2,3,14,...,0.0,0.0,0.0,0.0,75.0,210.0,NaN,NaN,NaN,1.0
1,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,Thaddeus Luster,0.0,0.0,0.0,0,0,0,0,...,0.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,1.0
2,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,Jason DeLucia,0.0,5.0,4.0,0,3,5,20,...,0.0,0.0,0.0,0.0,72.0,210.0,NaN,NaN,NaN,1.0
3,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,Scott Baker,0.0,0.0,2.0,0,0,2,14,...,0.0,0.0,1.0,1.0,71.0,190.0,NaN,Southpaw,24.0,1.0
4,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,David Levicki,0.0,0.0,0.0,0,4,5,95,...,1.0,1.0,9.0,15.0,72.0,210.0,NaN,Orthodox,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13853,UFC Fight Night: Lewis vs. Spivac,Tatsuro Taira vs. Jesus Aguilar,Tatsuro Taira,0.0,2.0,0.0,229,1,1,5,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0
13854,UFC Fight Night: Lewis vs. Spivac,Toshiomi Kazama vs. Rinya Nakamura,Rinya Nakamura,2.0,0.0,0.0,3,11,25,11,...,0.0,0.0,0.0,0.0,NaN,135.0,NaN,NaN,25.0,1.0
13855,UFC Fight Night: Lewis vs. Spivac,Toshiomi Kazama vs. Rinya Nakamura,Toshiomi Kazama,0.0,0.0,0.0,0,4,10,4,...,0.0,0.0,1.0,1.0,NaN,135.0,NaN,NaN,27.0,1.0
13856,UFC Fight Night: Lewis vs. Spivac,Yusaku Kinoshita vs. Adam Fugitt,Adam Fugitt,0.0,0.0,0.0,173,21,33,33,...,0.0,0.0,0.0,0.0,72.0,170.0,NaN,NaN,22.0,1.0


In [104]:
df_total.isna().any()

EVENT               False
BOUT                False
FIGHTER             False
KD_fighter          False
SUB.ATT_fighter     False
                    ...  
WEIGHT_opponent      True
REACH_opponent       True
STANCE_opponent      True
AGE_opponent         True
fightNo_opponent     True
Length: 61, dtype: bool

In [105]:
df_total = df_total.merge(df_results, on=['EVENT','BOUT'], how='left')

In [106]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13860 entries, 0 to 13859
Data columns (total 74 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   EVENT                        13860 non-null  object        
 1   BOUT                         13860 non-null  object        
 2   FIGHTER                      13860 non-null  object        
 3   KD_fighter                   13860 non-null  float64       
 4   SUB.ATT_fighter              13860 non-null  float64       
 5   REV._fighter                 13860 non-null  float64       
 6   CTRL_fighter                 13860 non-null  int32         
 7   SIG.STR._landed_fighter      13860 non-null  int32         
 8   SIG.STR._attemps_fighter     13860 non-null  int32         
 9   TOTAL STR._landed_fighter    13860 non-null  int32         
 10  TOTAL STR._attemps_fighter   13860 non-null  int32         
 11  TD_landed_fighter            13860 non-nu

In [107]:
df_total.isna().any()

EVENT              False
BOUT               False
FIGHTER            False
KD_fighter         False
SUB.ATT_fighter    False
                   ...  
TIME               False
TotalTime          False
REFEREE             True
DETAILS             True
URL                False
Length: 74, dtype: bool

In [108]:
df_total.loc[:,'Win'] = (df_total['Winner'].str.strip() == df_total['FIGHTER'].str.strip()).astype(int)
df_total.loc[:,'DRAW'] = (df_total['Winner'].str.strip() == 'Draw').astype(int)
df_total.loc[:,'No_contest'] = (df_total['Winner'].str.strip() == 'NC').astype(int)


In [109]:
df_total= df_total[['EVENT', 'BOUT','DATE_fighter','WeightClass', 'Tittle', 'Women_fight',
       'METHOD', 'TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS','URL','FIGHTER','Win', 'DRAW', 'No_contest', 'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter',
       'STANCE_fighter', 'AGE_fighter', 'fightNo_fighter', 'OPPONENT', 'HEIGHT_opponent',
       'WEIGHT_opponent', 'REACH_opponent', 'STANCE_opponent', 'AGE_opponent',
       'fightNo_opponent', 'KD_fighter', 'SUB.ATT_fighter',
       'REV._fighter', 'CTRL_fighter', 'SIG.STR._landed_fighter',
       'SIG.STR._attemps_fighter', 'TOTAL STR._landed_fighter',
       'TOTAL STR._attemps_fighter', 'TD_landed_fighter', 'TD_attemps_fighter',
       'HEAD_landed_fighter', 'HEAD_attemps_fighter', 'BODY_landed_fighter',
       'BODY_attemps_fighter', 'LEG_landed_fighter', 'LEG_attemps_fighter',
       'DISTANCE_landed_fighter', 'DISTANCE_attemps_fighter',
       'CLINCH_landed_fighter', 'CLINCH_attemps_fighter',
       'GROUND_landed_fighter', 'GROUND_attemps_fighter', 'KD_opponent',
       'SUB.ATT_opponent', 'REV._opponent', 'CTRL_opponent',
       'SIG.STR._landed_opponent', 'SIG.STR._attemps_opponent',
       'TOTAL STR._landed_opponent', 'TOTAL STR._attemps_opponent',
       'TD_landed_opponent', 'TD_attemps_opponent', 'HEAD_landed_opponent',
       'HEAD_attemps_opponent', 'BODY_landed_opponent',
       'BODY_attemps_opponent', 'LEG_landed_opponent', 'LEG_attemps_opponent',
       'DISTANCE_landed_opponent', 'DISTANCE_attemps_opponent',
       'CLINCH_landed_opponent', 'CLINCH_attemps_opponent',
       'GROUND_landed_opponent', 'GROUND_attemps_opponent']]

In [110]:
df_total.head()

,EVENT,BOUT,DATE_fighter,WeightClass,Tittle,Women_fight,METHOD,TIME FORMAT,ROUND,TIME,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
0,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,1994-03-11,Open Weight,0,0,Submission,[0],1,4:52,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,UFC 2: No Way Out,Frank Hamaker vs. Thaddeus Luster,1994-03-11,Open Weight,0,0,Submission,[0],1,4:52,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0
2,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,1994-03-11,Open Weight,0,0,Submission,[0],1,6:41,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
3,UFC 2: No Way Out,Jason DeLucia vs. Scott Baker,1994-03-11,Open Weight,0,0,Submission,[0],1,6:41,...,1.0,2.0,1.0,1.0,2.0,4.0,0.0,0.0,1.0,1.0
4,UFC 2: No Way Out,Johnny Rhodes vs. David Levicki,1994-03-11,Open Weight,0,0,KO/TKO,[0],1,12:13,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,9.0,15.0


In [111]:
df_total['METHOD'] = df_total['METHOD'].apply(lambda x: 'Decision' if 'Decision' in x else x.strip())
#df_total['METHOD'] = df_total['METHOD'].apply(lambda x: 'KO/TKO' if 'TKO' in x else x.strip())

In [112]:
ground = ['submission', 'guard', 'control', 'mount', 'crucifix']
clinch = ['clinch']
a = 'Punches to Head From GuardSubmission to Strikes'

In [113]:
np.sum([1 if x in a.lower() else 0 for x in ground])

2

In [114]:
def split_TKO(detail):
  lower = detail.lower()
  ground = ['submission', 'guard', 'control', 'mount', 'crucifix', 'ground']
  distance = ['kick', 'distance', 'fist', 'knee', 'elbow']
  clinch = ['clinch']
  injury = ['injury']
  if np.sum([1 if x in lower else 0 for x in ground]) > 0:
    return 'KO/TKO - ground'    
  elif np.sum([1 if x in lower else 0 for x in clinch]) > 0:
    return 'KO/TKO - clinch'
  elif np.sum([1 if x in lower else 0 for x in distance]) > 0:
    return 'KO/TKO - distance'
  elif np.sum([1 if x in lower else 0 for x in injury]) > 0:
    return 'KO/TKO - injury'
  else:
    return 'KO/TKO - other'

In [115]:
split_TKO('Punches to Head From GuardSubmission to Strikes')

'KO/TKO - ground'

In [116]:
df_total.columns

Index(['EVENT', 'BOUT', 'DATE_fighter', 'WeightClass', 'Tittle', 'Women_fight',
       'METHOD', 'TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS', 'URL', 'FIGHTER', 'Win', 'DRAW', 'No_contest',
       'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter', 'STANCE_fighter',
       'AGE_fighter', 'fightNo_fighter', 'OPPONENT', 'HEIGHT_opponent',
       'WEIGHT_opponent', 'REACH_opponent', 'STANCE_opponent', 'AGE_opponent',
       'fightNo_opponent', 'KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_la

In [117]:
df_total['METHOD'].unique()

array(['Submission', 'KO/TKO', "TKO - Doctor's Stoppage", 'Other',
       'Decision', 'Overturned', 'DQ', 'Could Not Continue'], dtype=object)

In [118]:
df_total['DETAILS']

0                               Keylock From Half Guard 
1                               Keylock From Half Guard 
2                             Triangle Choke From Mount 
3                             Triangle Choke From Mount 
4        Punches to Head From GuardSubmission to Strikes
                              ...                       
13855                 Triangle Armbar From Bottom Guard 
13856                         Punch to Head At Distance 
13857                         Punch to Head At Distance 
13858                         Elbows to Head From Mount 
13859                         Elbows to Head From Mount 
Name: DETAILS, Length: 13860, dtype: object

In [119]:
df_total['METHOD'] = df_total.apply(lambda row: split_TKO(row['DETAILS']) if row['METHOD'] =='KO/TKO' else row['METHOD'], axis=1)

In [120]:
df_total[df_total['METHOD']=='KO/TKO - other']['DETAILS'].unique()

array(['toCorner Stoppage', 'toFatigue',
       'toReferee stoppage between rounds', 'Punch to Head From Bottom ',
       'to ', 'toReferee Stoppage Between Rounds',
       'toStoppage Between Rounds'], dtype=object)

In [121]:
df_total.loc[df_total['METHOD'].str.contains('KO'),['METHOD', 'DETAILS']].tail(50)

,METHOD,DETAILS
13738,KO/TKO - distance,Punches to Head At Distance
13739,KO/TKO - distance,Punches to Head At Distance
13740,KO/TKO - clinch,Punch to Head In Clinch
13741,KO/TKO - clinch,Punch to Head In Clinch
13744,KO/TKO - ground,Punches to Head On Ground
13745,KO/TKO - ground,Punches to Head On Ground
13746,KO/TKO - distance,Punch to Head At Distance
13747,KO/TKO - distance,Punch to Head At Distance
13754,KO/TKO - distance,Punches to Head At Distance
13755,KO/TKO - distance,Punches to Head At Distance


In [122]:
df_total = df_total.rename(columns={"DATE_fighter": "DATE_Event"})

In [123]:
df_total['REACH_fighter'].unique()

array([nan, 72., 75., 74., 76., 70., 73., 77., 68., 78., 67., 79., 69.,
       80., 71., 64., 66., 82., 81., 84., 63., 65., 62., 60., 83., 61.,
       58., 59.])

In [124]:
df_total.iloc[:-5,18:23]

,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,AGE_fighter
0,NaN,NaN,NaN,NaN,NaN
1,75.0,210.0,NaN,NaN,NaN
2,71.0,190.0,NaN,Southpaw,24.0
3,72.0,210.0,NaN,NaN,NaN
4,77.0,275.0,NaN,NaN,NaN
...,...,...,...,...,...
13850,71.0,250.0,73.0,Southpaw,36.0
13851,75.0,249.0,78.0,Orthodox,37.0
13852,NaN,125.0,NaN,NaN,NaN
13853,NaN,125.0,NaN,NaN,NaN


In [125]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13860 entries, 0 to 13859
Data columns (total 75 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   EVENT                        13860 non-null  object        
 1   BOUT                         13860 non-null  object        
 2   DATE_Event                   13860 non-null  datetime64[ns]
 3   WeightClass                  13860 non-null  object        
 4   Tittle                       13860 non-null  int64         
 5   Women_fight                  13860 non-null  int64         
 6   METHOD                       13860 non-null  object        
 7   TIME FORMAT                  13860 non-null  object        
 8   ROUND                        13860 non-null  int64         
 9   TIME                         13860 non-null  object        
 10  TotalTime                    13860 non-null  int64         
 11  REFEREE                      13796 non-nu

In [126]:
df_total.isna().sum()

EVENT                        0
BOUT                         0
DATE_Event                   0
WeightClass                  0
Tittle                       0
                            ..
DISTANCE_attemps_opponent    9
CLINCH_landed_opponent       9
CLINCH_attemps_opponent      9
GROUND_landed_opponent       9
GROUND_attemps_opponent      9
Length: 75, dtype: int64

In [127]:
df_total.columns

Index(['EVENT', 'BOUT', 'DATE_Event', 'WeightClass', 'Tittle', 'Women_fight',
       'METHOD', 'TIME FORMAT', 'ROUND', 'TIME', 'TotalTime', 'REFEREE',
       'DETAILS', 'URL', 'FIGHTER', 'Win', 'DRAW', 'No_contest',
       'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter', 'STANCE_fighter',
       'AGE_fighter', 'fightNo_fighter', 'OPPONENT', 'HEIGHT_opponent',
       'WEIGHT_opponent', 'REACH_opponent', 'STANCE_opponent', 'AGE_opponent',
       'fightNo_opponent', 'KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_land

In [128]:
cols_to_sum = ['Win', 'DRAW', 'No_contest', 'TotalTime','KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_landed_fighter',
       'GROUND_attemps_fighter', 'KD_opponent', 'SUB.ATT_opponent',
       'REV._opponent', 'CTRL_opponent', 'SIG.STR._landed_opponent',
       'SIG.STR._attemps_opponent', 'TOTAL STR._landed_opponent',
       'TOTAL STR._attemps_opponent', 'TD_landed_opponent',
       'TD_attemps_opponent', 'HEAD_landed_opponent', 'HEAD_attemps_opponent',
       'BODY_landed_opponent', 'BODY_attemps_opponent', 'LEG_landed_opponent',
       'LEG_attemps_opponent', 'DISTANCE_landed_opponent',
       'DISTANCE_attemps_opponent', 'CLINCH_landed_opponent',
       'CLINCH_attemps_opponent', 'GROUND_landed_opponent',
       'GROUND_attemps_opponent']

In [129]:
for col in list(df_total.columns):
  if '_fighter' in col:
    col_prefix = col.replace('_fighter','' )
    if not col == 'STANCE_fighter':
      print(col, col_prefix)
      df_total.loc[:,col_prefix+'_diff'] = df_total[col].astype(float) - df_total[col_prefix +'_opponent'].astype(float)

HEIGHT_fighter HEIGHT
WEIGHT_fighter WEIGHT
REACH_fighter REACH
AGE_fighter AGE
fightNo_fighter fightNo
KD_fighter KD
SUB.ATT_fighter SUB.ATT
REV._fighter REV.
CTRL_fighter CTRL
SIG.STR._landed_fighter SIG.STR._landed
SIG.STR._attemps_fighter SIG.STR._attemps
TOTAL STR._landed_fighter TOTAL STR._landed
TOTAL STR._attemps_fighter TOTAL STR._attemps
TD_landed_fighter TD_landed
TD_attemps_fighter TD_attemps
HEAD_landed_fighter HEAD_landed
HEAD_attemps_fighter HEAD_attemps
BODY_landed_fighter BODY_landed
BODY_attemps_fighter BODY_attemps
LEG_landed_fighter LEG_landed
LEG_attemps_fighter LEG_attemps
DISTANCE_landed_fighter DISTANCE_landed
DISTANCE_attemps_fighter DISTANCE_attemps
CLINCH_landed_fighter CLINCH_landed
CLINCH_attemps_fighter CLINCH_attemps
GROUND_landed_fighter GROUND_landed
GROUND_attemps_fighter GROUND_attemps


In [130]:
def forma(fighter_name, datum):
  vysledek=''
  skore = 0
  koef =0.5
  result = ['W' if x==1 else 'L' for x in df_total['Win'][(df_total['FIGHTER']==fighter_name) & (df_total['DATE_Event']<datum)]]
  for vyhra in result[:-6:-1]:
    if vyhra =='W':
      skore+=koef
    else:
      skore-=koef
    koef-=0.1  
    vysledek +=vyhra+' '
  vysledek=vysledek[:-1]  
  return (vysledek,skore)

In [131]:
df_total.loc[:,"form_skore_fighter"] = df_total.apply(lambda x:forma(x.FIGHTER, x.DATE_Event)[1], axis=1).astype(float)
df_total.loc[:,"form_skore_opponent"] = df_total.apply(lambda x:forma(x.OPPONENT, x.DATE_Event)[1], axis=1).astype(float)

In [132]:
df_total.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13860 entries, 0 to 13859
Data columns (total 104 columns):
 #    Column                       Non-Null Count  Dtype         
---   ------                       --------------  -----         
 0    EVENT                        13860 non-null  object        
 1    BOUT                         13860 non-null  object        
 2    DATE_Event                   13860 non-null  datetime64[ns]
 3    WeightClass                  13860 non-null  object        
 4    Tittle                       13860 non-null  int64         
 5    Women_fight                  13860 non-null  int64         
 6    METHOD                       13860 non-null  object        
 7    TIME FORMAT                  13860 non-null  object        
 8    ROUND                        13860 non-null  int64         
 9    TIME                         13860 non-null  object        
 10   TotalTime                    13860 non-null  int64         
 11   REFEREE                   

In [133]:
df_total[['FIGHTER', 'form_skore_fighter']].tail(10)

,FIGHTER,form_skore_fighter
13850,Blagoy Ivanov,0.1
13851,Marcin Tybura,0.7
13852,HyunSung Park,0.0
13853,SeungGuk Choi,0.0
13854,Jesus Aguilar,0.0
13855,Tatsuro Taira,0.9
13856,Rinya Nakamura,0.0
13857,Toshiomi Kazama,0.0
13858,Adam Fugitt,-0.5
13859,Yusaku Kinoshita,0.0


In [134]:
dummies = pd.get_dummies(df_total['METHOD'])
dummies['Other'] = dummies['Other'] + dummies['Could Not Continue'] + dummies['DQ'] + dummies['KO/TKO - injury'] + dummies['KO/TKO - other'] + dummies['Overturned'] + dummies['TKO - Doctor\'s Stoppage']
dummies = dummies.drop(['Could Not Continue', 'DQ', 'KO/TKO - injury', 'KO/TKO - other', 'Overturned','TKO - Doctor\'s Stoppage' ], axis = 1)
dummies['Win-striking'] = dummies['KO/TKO - distance'] + dummies['KO/TKO - clinch']
dummies['Win-ground'] = dummies['Submission'] + dummies['KO/TKO - ground']
dummies = dummies.drop(['KO/TKO - distance', 'KO/TKO - clinch', 'Submission','KO/TKO - ground'],axis=1)
dummies

,Decision,Other,Win-striking,Win-ground
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
13855,0,0,0,1
13856,0,0,1,0
13857,0,0,1,0
13858,0,0,0,1


In [135]:
df_method_win = df_total.loc[df_total['Win']==1,['FIGHTER', 'Win']].join(dummies).groupby('FIGHTER').sum().reset_index()
df_method_win = df_method_win.drop('Win',axis=1)
df_method_win.rename(columns = {'Decision':'Win_Decision', 'Other':'Win_Other'}, inplace = True)
df_method_win
df_method_losts = df_total.loc[df_total['Win']==0,['FIGHTER', 'Win']].join(dummies).groupby('FIGHTER').sum().reset_index()
df_method_losts = df_method_losts.drop('Win',axis=1)
df_method_losts.rename(columns = {'Decision':'Lost_Decision', 'Other':'Lost_Other', 'Win-striking':'Lost-striking', 'Win-ground': 'Lost-ground'}, inplace = True)
df_method_losts

,FIGHTER,Lost_Decision,Lost_Other,Lost-striking,Lost-ground
0,AJ Dobson,1,0,0,0
1,AJ Fletcher,2,0,0,0
2,Aalon Cruz,0,0,0,2
3,Aaron Brink,0,0,0,1
4,Aaron Phillips,2,0,0,1
...,...,...,...,...,...
2212,Zhang Lipeng,2,0,0,0
2213,Zhang Tiequan,2,0,1,0
2214,Zhang Weili,1,0,1,0
2215,Zubaira Tukhugov,3,0,0,0


In [136]:
len(df_total['FIGHTER'].unique())

2329

In [137]:
df_total = df_total.fillna(0)

In [138]:
fighters_total_df = df_total.pivot_table(index=['FIGHTER', 'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter', 'STANCE_fighter'], values=cols_to_sum, aggfunc=np.sum)[cols_to_sum].reset_index()

In [139]:
fighters_total_df

,FIGHTER,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,Win,DRAW,No_contest,TotalTime,KD_fighter,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
0,AJ Dobson,73.0,185.0,76.0,Orthodox,0,0,0,900,0.0,...,9.0,9.0,1.0,1.0,29.0,57.0,4.0,7.0,47.0,64.0
1,AJ Fletcher,70.0,170.0,67.0,Switch,0,0,0,1800,0.0,...,33.0,42.0,16.0,18.0,116.0,240.0,13.0,21.0,40.0,54.0
2,Aalon Cruz,72.0,145.0,78.0,Switch,0,0,0,185,0.0,...,1.0,2.0,2.0,2.0,9.0,14.0,7.0,8.0,49.0,70.0
3,Aaron Brink,75.0,205.0,0.0,Orthodox,0,0,0,55,0.0,...,0.0,0.0,1.0,1.0,2.0,2.0,0.0,2.0,0.0,0.0
4,Aaron Phillips,69.0,135.0,71.0,Southpaw,0,0,0,2249,0.0,...,21.0,30.0,10.0,13.0,25.0,60.0,31.0,40.0,73.0,113.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2325,Zhang Lipeng,71.0,155.0,71.0,Southpaw,2,0,0,3600,0.0,...,16.0,34.0,48.0,64.0,48.0,146.0,43.0,56.0,22.0,43.0
2326,Zhang Tiequan,68.0,155.0,69.0,Orthodox,1,0,0,2180,0.0,...,13.0,18.0,3.0,3.0,46.0,112.0,1.0,2.0,33.0,47.0
2327,Zhang Weili,64.0,115.0,63.0,Switch,7,0,0,5954,2.0,...,75.0,103.0,106.0,147.0,337.0,784.0,40.0,45.0,26.0,35.0
2328,Zubaira Tukhugov,68.0,145.0,68.0,Orthodox,5,0,0,5862,3.0,...,62.0,113.0,70.0,100.0,264.0,777.0,8.0,17.0,5.0,5.0


In [140]:
fighters_total_df['Fights'] = fighters_total_df['FIGHTER'].apply(lambda x: df_total['FIGHTER'].tolist().count(x))

In [141]:
fighters_total_df['Lost'] = fighters_total_df['Fights'] - fighters_total_df['Win'] - fighters_total_df['DRAW'] - fighters_total_df['No_contest']

In [142]:
fighters_total_df = fighters_total_df[['FIGHTER', 'HEIGHT_fighter', 'WEIGHT_fighter', 'REACH_fighter',
       'STANCE_fighter', 'Fights', 'Win', 'DRAW', 'Lost','No_contest' , 'TotalTime',
       'KD_fighter', 'SUB.ATT_fighter', 'REV._fighter', 'CTRL_fighter',
       'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_landed_fighter',
       'GROUND_attemps_fighter', 'KD_opponent', 'SUB.ATT_opponent',
       'REV._opponent', 'CTRL_opponent', 'SIG.STR._landed_opponent',
       'SIG.STR._attemps_opponent', 'TOTAL STR._landed_opponent',
       'TOTAL STR._attemps_opponent', 'TD_landed_opponent',
       'TD_attemps_opponent', 'HEAD_landed_opponent', 'HEAD_attemps_opponent',
       'BODY_landed_opponent', 'BODY_attemps_opponent', 'LEG_landed_opponent',
       'LEG_attemps_opponent', 'DISTANCE_landed_opponent',
       'DISTANCE_attemps_opponent', 'CLINCH_landed_opponent',
       'CLINCH_attemps_opponent', 'GROUND_landed_opponent',
       'GROUND_attemps_opponent']]

In [143]:
fighters_total_df.sort_values('Fights', ascending=False)

,FIGHTER,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,Fights,Win,DRAW,Lost,No_contest,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
1044,Jim Miller,68.0,155.0,71.0,Southpaw,40,24,0,15,1,...,244.0,343.0,111.0,148.0,799.0,2238.0,162.0,241.0,174.0,281.0
130,Andrei Arlovski,75.0,240.0,77.0,Orthodox,39,23,0,15,1,...,204.0,297.0,223.0,256.0,904.0,2329.0,156.0,242.0,158.0,235.0
629,Donald Cerrone,73.0,170.0,73.0,Orthodox,38,23,0,14,1,...,444.0,668.0,312.0,379.0,1558.0,3375.0,211.0,305.0,56.0,88.0
1018,Jeremy Stephens,69.0,145.0,71.0,Orthodox,34,15,0,18,1,...,210.0,307.0,261.0,316.0,939.0,2448.0,108.0,158.0,142.0,228.0
430,Clay Guida,67.0,155.0,70.0,Orthodox,34,18,0,16,0,...,195.0,277.0,92.0,115.0,847.0,2643.0,189.0,263.0,114.0,138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1310,Kleydson Rodrigues,66.0,125.0,67.0,Orthodox,1,0,0,1,0,...,18.0,25.0,11.0,13.0,45.0,92.0,7.0,10.0,19.0,25.0
1312,Koji Kitao,79.0,350.0,0.0,Southpaw,1,0,0,1,0,...,0.0,0.0,1.0,1.0,2.0,2.0,0.0,0.0,1.0,1.0
1317,Kristian Rothaermel,72.0,205.0,0.0,Open Stance,1,0,0,1,0,...,0.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0,9.0,13.0
1318,Kristof Midoux,73.0,220.0,0.0,Orthodox,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,25.0,49.0


In [144]:
fighters_total_df.loc[:,"form_skore_fighter"] = fighters_total_df.apply(lambda x:forma(x.FIGHTER, df_total["DATE_Event"].max())[1], axis=1).astype(float)

C:\Users\spravce\AppData\Local\Temp\ipykernel_81028\2484977330.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fighters_total_df.loc[:,"form_skore_fighter"] = fighters_total_df.apply(lambda x:forma(x.FIGHTER, df_total["DATE_Event"].max())[1], axis=1).astype(float)


In [145]:
fighters_total_df.tail()

,FIGHTER,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,Fights,Win,DRAW,Lost,No_contest,...,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent,form_skore_fighter
2325,Zhang Lipeng,71.0,155.0,71.0,Southpaw,4,2,0,2,0,...,34.0,48.0,64.0,48.0,146.0,43.0,56.0,22.0,43.0,-0.4
2326,Zhang Tiequan,68.0,155.0,69.0,Orthodox,4,1,0,3,0,...,18.0,3.0,3.0,46.0,112.0,1.0,2.0,33.0,47.0,-1.0
2327,Zhang Weili,64.0,115.0,63.0,Switch,9,7,0,2,0,...,103.0,106.0,147.0,337.0,784.0,40.0,45.0,26.0,35.0,0.5
2328,Zubaira Tukhugov,68.0,145.0,68.0,Orthodox,8,5,0,3,0,...,113.0,70.0,100.0,264.0,777.0,8.0,17.0,5.0,5.0,0.1
2329,Zviad Lazishvili,66.0,135.0,69.0,Orthodox,1,0,0,1,0,...,19.0,13.0,15.0,92.0,188.0,0.0,0.0,0.0,0.0,-0.5


In [146]:
from datetime import date
today = date.today()
df_skills = fighters_total_df.copy()
df_skills = df_skills.merge(df_method_win,on='FIGHTER', how='left')

In [147]:
df_skills = df_skills.merge(df_method_losts,on='FIGHTER', how='left')
from datetime import date

today = date.today()
df_fighter_tott['AGE'] = (pd.to_datetime(today) - pd.to_datetime(df_fighter_tott['DOB'])).astype('<m8[Y]')
df_skills = df_skills.merge(df_fighter_tott[['FIGHTER','AGE']], on='FIGHTER', how='left')
df_skills = df_skills.fillna(0)

In [148]:
df_skills

,FIGHTER,HEIGHT_fighter,WEIGHT_fighter,REACH_fighter,STANCE_fighter,Fights,Win,DRAW,Lost,No_contest,...,form_skore_fighter,Win_Decision,Win_Other,Win-striking,Win-ground,Lost_Decision,Lost_Other,Lost-striking,Lost-ground,AGE
0,AJ Dobson,73.0,185.0,76.0,Orthodox,1,0,0,1,0,...,-0.5,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,31.0
1,AJ Fletcher,70.0,170.0,67.0,Switch,2,0,0,2,0,...,-0.9,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,25.0
2,Aalon Cruz,72.0,145.0,78.0,Switch,2,0,0,2,0,...,-0.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,33.0
3,Aaron Brink,75.0,205.0,0.0,Orthodox,1,0,0,1,0,...,-0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,48.0
4,Aaron Phillips,69.0,135.0,71.0,Southpaw,3,0,0,3,0,...,-1.2,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,Zhang Lipeng,71.0,155.0,71.0,Southpaw,4,2,0,2,0,...,-0.4,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,32.0
2328,Zhang Tiequan,68.0,155.0,69.0,Orthodox,4,1,0,3,0,...,-1.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,44.0
2329,Zhang Weili,64.0,115.0,63.0,Switch,9,7,0,2,0,...,0.5,3.0,0.0,2.0,2.0,1.0,0.0,1.0,0.0,33.0
2330,Zubaira Tukhugov,68.0,145.0,68.0,Orthodox,8,5,0,3,0,...,0.1,3.0,0.0,1.0,1.0,3.0,0.0,0.0,0.0,32.0


In [149]:
percentil_fight =4
df_skills['fights_coef'] = (df_skills['Fights'] - percentil_fight)/ (df_skills['Lost-ground']+1)
df_skills['lost_ground_rat'] = (df_skills['Lost-ground']/df_skills['Fights'])
df_skills['coef_zada'] = (df_skills['CTRL_opponent']/df_skills['TotalTime'])
df_skills['ground_def_skill'] = (30*(1-df_skills['coef_zada']))+(30*(1-df_skills['lost_ground_rat']))+df_skills['fights_coef']+10

df_skills['fights_coef_gr_att'] = (df_skills['Fights'] - percentil_fight)/ (df_skills['Fights']  - df_skills['Win-ground']+1)
df_skills['win_ground_rat'] = (df_skills['Win-ground']/df_skills['Fights'])
df_skills['coef_kontrola'] = (df_skills['CTRL_fighter']/df_skills['TotalTime'])
df_skills['ground_att_skill'] = (45*df_skills['coef_kontrola'])+(45*(df_skills['win_ground_rat']))+12*df_skills['fights_coef_gr_att']+20

df_skills['fights_coef_str_def'] = (df_skills['Fights'])/ (df_skills['Lost-striking']+1)
df_skills['lost_str_rat'] = (df_skills['Lost-striking']/df_skills['Fights'])
df_skills['coef_nohy'] = (df_skills['TotalTime'] - df_skills['CTRL_fighter'] - df_skills['CTRL_opponent'])/df_skills['TotalTime']
df_skills['stand_def_skill'] = (40*(df_skills['coef_nohy']))+(40*(1-df_skills['lost_str_rat']))+df_skills['fights_coef_str_def']

df_skills['fights_coef_str_att'] = (df_skills['Fights'] - percentil_fight)/ (df_skills['Fights']  - df_skills['Win-striking']+1)
df_skills['win_ground_rat'] = (df_skills['Win-striking']/df_skills['Fights'])
df_skills['coef_nohy'] = (df_skills['TotalTime'] - df_skills['CTRL_fighter'] - df_skills['CTRL_opponent'])/df_skills['TotalTime']
df_skills['stand_att_skill'] = (45*df_skills['coef_nohy'])+(45*(df_skills['win_ground_rat']))+12*df_skills['fights_coef_str_att']+20

df_skills['fights_coef_stamina'] = (df_skills['Fights']-4)/ (df_skills['Lost']+1)
df_skills['avg_time'] = (df_skills['TotalTime']/df_skills['Fights']/300)
df_skills['win_dec_rat'] = (df_skills['Win_Decision']/df_skills['Fights'])
df_skills['stamina'] = 5*df_skills['fights_coef_stamina'] + df_skills['avg_time'] +40*df_skills['win_dec_rat']+25



# **df_pro_model**

In [150]:
cols_to_sum = ['Win', 'DRAW', 'No_contest', 'TotalTime','KD_fighter', 'SUB.ATT_fighter', 'REV._fighter',
       'CTRL_fighter', 'SIG.STR._landed_fighter', 'SIG.STR._attemps_fighter',
       'TOTAL STR._landed_fighter', 'TOTAL STR._attemps_fighter',
       'TD_landed_fighter', 'TD_attemps_fighter', 'HEAD_landed_fighter',
       'HEAD_attemps_fighter', 'BODY_landed_fighter', 'BODY_attemps_fighter',
       'LEG_landed_fighter', 'LEG_attemps_fighter', 'DISTANCE_landed_fighter',
       'DISTANCE_attemps_fighter', 'CLINCH_landed_fighter',
       'CLINCH_attemps_fighter', 'GROUND_landed_fighter',
       'GROUND_attemps_fighter', 'KD_opponent', 'SUB.ATT_opponent',
       'REV._opponent', 'CTRL_opponent', 'SIG.STR._landed_opponent',
       'SIG.STR._attemps_opponent', 'TOTAL STR._landed_opponent',
       'TOTAL STR._attemps_opponent', 'TD_landed_opponent',
       'TD_attemps_opponent', 'HEAD_landed_opponent', 'HEAD_attemps_opponent',
       'BODY_landed_opponent', 'BODY_attemps_opponent', 'LEG_landed_opponent',
       'LEG_attemps_opponent', 'DISTANCE_landed_opponent',
       'DISTANCE_attemps_opponent', 'CLINCH_landed_opponent',
       'CLINCH_attemps_opponent', 'GROUND_landed_opponent',
       'GROUND_attemps_opponent']

In [151]:
df_total.groupby('FIGHTER')[cols_to_sum].transform(lambda x: x.shift(1).cumsum())

,Win,DRAW,No_contest,TotalTime,KD_fighter,SUB.ATT_fighter,REV._fighter,CTRL_fighter,SIG.STR._landed_fighter,SIG.STR._attemps_fighter,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13855,2.0,0.0,0.0,1459.0,1.0,5.0,2.0,796.0,79.0,123.0,...,8.0,12.0,1.0,2.0,21.0,57.0,3.0,3.0,4.0,4.0
13856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13858,0.0,0.0,0.0,669.0,0.0,0.0,0.0,111.0,64.0,113.0,...,7.0,9.0,2.0,2.0,64.0,127.0,12.0,17.0,7.0,7.0


In [152]:
df_pre_comp = df_total[['EVENT','BOUT', 'FIGHTER','OPPONENT','METHOD','Win','Women_fight','ROUND','HEIGHT_diff','REACH_diff','AGE_diff','form_skore_fighter', 'form_skore_opponent','fightNo_fighter','fightNo_opponent']]

In [153]:
df_pre_comp = df_pre_comp.join(df_total.groupby('FIGHTER')[cols_to_sum].transform(lambda x: x.shift(1).cumsum()),rsuffix='_tot')

In [154]:
df_pre_comp.tail()

,EVENT,BOUT,FIGHTER,OPPONENT,METHOD,Win,Women_fight,ROUND,HEIGHT_diff,REACH_diff,...,BODY_landed_opponent,BODY_attemps_opponent,LEG_landed_opponent,LEG_attemps_opponent,DISTANCE_landed_opponent,DISTANCE_attemps_opponent,CLINCH_landed_opponent,CLINCH_attemps_opponent,GROUND_landed_opponent,GROUND_attemps_opponent
13855,UFC Fight Night: Lewis vs. Spivac,Tatsuro Taira vs. Jesus Aguilar,Tatsuro Taira,Jesus Aguilar,Submission,1,0,1,0.0,0.0,...,8.0,12.0,1.0,2.0,21.0,57.0,3.0,3.0,4.0,4.0
13856,UFC Fight Night: Lewis vs. Spivac,Toshiomi Kazama vs. Rinya Nakamura,Rinya Nakamura,Toshiomi Kazama,KO/TKO - distance,1,0,1,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13857,UFC Fight Night: Lewis vs. Spivac,Toshiomi Kazama vs. Rinya Nakamura,Toshiomi Kazama,Rinya Nakamura,KO/TKO - distance,0,0,1,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13858,UFC Fight Night: Lewis vs. Spivac,Yusaku Kinoshita vs. Adam Fugitt,Adam Fugitt,Yusaku Kinoshita,KO/TKO - ground,1,0,1,1.0,0.0,...,7.0,9.0,2.0,2.0,64.0,127.0,12.0,17.0,7.0,7.0
13859,UFC Fight Night: Lewis vs. Spivac,Yusaku Kinoshita vs. Adam Fugitt,Yusaku Kinoshita,Adam Fugitt,KO/TKO - ground,0,0,1,-1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
df_pre_comp[df_pre_comp['FIGHTER']=='Conor McGregor']['ROUND']

4448     1
4704     3
5604     1
5792     1
6080     2
6533     2
6962     1
7156     2
7574     5
7772     2
9582     4
10890    1
11850    2
12310    1
Name: ROUND, dtype: int64

In [156]:
df_total[df_total['FIGHTER']=='David Abbott'].isna().sum().sort_values()

EVENT                      0
HEIGHT_diff                0
GROUND_attemps_opponent    0
GROUND_landed_opponent     0
CLINCH_attemps_opponent    0
                          ..
AGE_opponent               0
STANCE_opponent            0
REACH_opponent             0
form_skore_fighter         0
form_skore_opponent        0
Length: 104, dtype: int64

In [157]:
df_fighter_tott.to_csv('fighter_details.csv',index=False)
fighters_total_df.to_csv('fighter_total_stats.csv',index=False)
df_total.to_csv('fight_with_stats.csv',index=False)
df_pre_comp.to_csv('fight_with_stats_precomp.csv',index=False)
df_results.to_csv('fight_result.csv',index=False)
df_skills.to_csv('df_skills.csv',index=False)